In [1]:
!pip install gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 76.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.8 MB/s eta 0:00:00


In [ ]:
import gradio as gr
import pandas as pd
import time
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from peft import PeftModel, PeftConfig

adapter_model_id = "Martinamousa95/bart_model_fine_tuning"
config = PeftConfig.from_pretrained(adapter_model_id)
base_model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(base_model, adapter_model_id)
tokenizer = AutoTokenizer.from_pretrained(adapter_model_id)

def generate_structured_output(text, max_new_tokens=128):
    # Directly pass the input text as the prompt
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        num_beams=4,
        no_repeat_ngram_size=3,
        early_stopping=True
    )

    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary

def format_summary(summary):
    sentences = summary.split(". ")
    formatted_summary = "<br>".join([sentence.strip() + "." for sentence in sentences if sentence.strip()])
    return formatted_summary

def gradio_interface(legal_text):
    start_time = time.time()  

    summary = generate_structured_output(legal_text)

    if not summary:
        return "No summary generated. Please try again with a different text."

    formatted_summary = format_summary(summary)

    end_time = time.time()  
    time_taken = round(end_time - start_time, 2)  
    formatted_summary += f"<br><br><b>Time taken for response: {time_taken} seconds</b>"

    return formatted_summary

iface = gr.Interface(
    fn=gradio_interface,
    inputs="text",
    outputs="html",  
    title="Legal Text Summarizer",
    description="Provide a legal text to generate a structured summary with each sentence in a new line, along with the time taken for response."
)

# Launch the Gradio app
iface.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f992db083217f9108c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
